In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
import torchreid


D:\Drone_object_tracking\torchreid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [3]:
from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000013_00000_v import VisDroneChild_uav0000013_00000_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000013_00000_v', VisDroneChild_uav0000013_00000_v)

In [4]:
from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000013_01073_v import VisDroneChild_uav0000013_01073_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000013_01073_v', VisDroneChild_uav0000013_01073_v)


In [5]:

from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000086_00000_v import VisDroneChild_uav0000086_00000_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000086_00000_v', VisDroneChild_uav0000086_00000_v)


In [6]:

from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000137_00458_v import VisDroneChild_uav0000137_00458_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000137_00458_v', VisDroneChild_uav0000137_00458_v)


In [7]:

from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000182_00000_v import VisDroneChild_uav0000182_00000_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000182_00000_v', VisDroneChild_uav0000182_00000_v)


In [8]:
datamanager = torchreid.data.VideoDataManager(
    root='D:\\Drone_object_tracking\\reid-data\\VisDrone2019-MOT-val',
    sources=['VisDroneChild_uav0000182_00000_v'],
    targets = ['VisDroneChild_uav0000137_00458_v'],
    height = 200,    #resizes images to this
    width=100,
    batch_size_train = 9,#how many tracklets in a batch. Small batch size may cause error because 1 id is nor enough to form tripleet loss. Batch_size >=  num_instances*3
    batch_size_test = 9,#how many tracklets in a batch
    seq_len = 10,    # how many frmaes in a tracklet
    num_instances = 3, #how many tracklets per id in a batch
    train_sampler='CustomDatasetSampler'
)

Building train transforms ...
+ resize to 200x100
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 200x100
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
self.video_name uav0000182_00000_v
=> Loaded VisDroneChild_uav0000182_00000_v
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   151 |        1156 |         1
  query    |    91 |         192 |         1
  gallery  |   148 |         964 |         1
  -------------------------------------------
=> Loading test (target) dataset
self.video_name uav0000137_00458_v
=> Loaded VisDroneChild_uav0000137_00458_v
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -----------------------------------------

In [9]:
model = torchreid.models.build_model(
    name='mobilenetv2_x1_4',
    num_classes=datamanager.max_train_pid+1,#КОСТЫЛЬ, возможно)
    loss='triplet',
    pretrained=False
    
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=300,
    gamma = 1
)


In [10]:
engine = torchreid.engine.VideoTripletEngine(
    datamanager, model,optimizer,margin=0.5, weight_t=1, weight_x=0,scheduler=scheduler,
    pooling_method='avg'
)

In [11]:
"""engine = torchreid.engine.VideoSoftmaxEngine(
    datamanager, model,optimizer,scheduler=scheduler,
    pooling_method='avg'
)"""

"engine = torchreid.engine.VideoSoftmaxEngine(\n    datamanager, model,optimizer,scheduler=scheduler,\n    pooling_method='avg'\n)"

In [12]:


engine.run(
    max_epoch=150,
    save_dir='log/mobilenetv2_x1_4-visdrone',
    print_freq=10,  #if (self.batch_idx + 1) % print_freq == 0:, where for self.batch_idx, data in enumerate(self.train_loader):
    eval_freq=1,
    #start_epoch=start_epoch,
    #test_only=True,
    #visrank=True
    #start_epoch=start_epoch,
    
)
#loss_t - triplet loss
#loss_x - cross entropy loss
#validation on same as training - 100% results
#sampler selects tracklets to batches??
""" used like:
self.train_loader = torch.utils.data.DataLoader(
            trainset,
            sampler=build_train_sampler(
                trainset.train,
                train_sampler,
                batch_size=batch_size_train,
                num_instances=num_instances,
                num_cams=num_cams,
                num_datasets=num_datasets
            ),
            ...
        )
"""

"""
Если вообще не учить перед валидацией - 95+
Если хотябы одну эпоху на любом датасете будет хуже
В QUERY и GALLERY одну и тоже может попасть
"""
#HARD MINE YRIPLET LOSS distAP not considered


=> Start training
epoch: [1/150][1/122]	time 28.170 (28.170)	data 22.096 (22.096)	eta 5 days, 23:11:26	loss_t 0.7705 (0.7705)	lr 0.000300
epoch: [1/150][2/122]	time 3.199 (15.685)	data 0.031 (11.064)	eta 3 days, 7:43:15	loss_t 0.9393 (0.8549)	lr 0.000300
epoch: [1/150][3/122]	time 5.579 (12.316)	data 0.000 (7.376)	eta 2 days, 14:35:44	loss_t 0.6761 (0.7953)	lr 0.000300
epoch: [1/150][4/122]	time 5.879 (10.707)	data 0.000 (5.532)	eta 2 days, 6:24:50	loss_t 0.3353 (0.6803)	lr 0.000300
epoch: [1/150][5/122]	time 5.774 (9.720)	data 0.000 (4.425)	eta 2 days, 1:23:51	loss_t 0.9036 (0.7250)	lr 0.000300
epoch: [1/150][6/122]	time 6.146 (9.124)	data 0.000 (3.688)	eta 1 day, 22:22:03	loss_t 2.0488 (0.9456)	lr 0.000300
epoch: [1/150][7/122]	time 5.959 (8.672)	data 0.000 (3.161)	eta 1 day, 20:04:02	loss_t 0.6592 (0.9047)	lr 0.000300
epoch: [1/150][8/122]	time 5.930 (8.330)	data 0.000 (2.766)	eta 1 day, 18:19:24	loss_t 1.8661 (1.0249)	lr 0.000300
epoch: [1/150][9/122]	time 5.940 (8.064)	data 0.000 

KeyboardInterrupt: 